In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from datetime import date, datetime
from sklearn.impute import SimpleImputer

## 1) Exploratory Data Analysis

There is one extremely large outlier in the training data (price = \$99998) that I excluded because it was severly skewing my model. The price column and acceptance rate columns had unnecessary characters so in my cleaning I removed those and converted to numeric values. Along with that, I transformed the date columns to numeric by calculated "months since" each respective date. True/False columns were converted to 1/0. There were many values for neighbourhood so I grouped any neighbourhood with less than 200 value counts into 'Other'.

## 2) Data Cleaning/Preparation

In [2]:
raw_train = pd.read_csv('datasets/train_regression.csv')
raw_test = pd.read_csv('datasets/test_regression.csv')

In [3]:
# Clean and process the data
train = raw_train.copy()
test = raw_test.copy()


# Process and convert price to float
train['price'] = train['price'].str.replace(',', '').str.replace('$', '', regex=False).astype(float)


# Process acceptance rates
train['host_acceptance_rate'] = train['host_acceptance_rate'].str.replace('%', '').astype(float) / 100
train['host_response_rate'] = train['host_response_rate'].str.replace('%', '').astype(float) / 100

test['host_acceptance_rate'] = test['host_acceptance_rate'].str.replace('%', '').astype(float) / 100
test['host_response_rate'] = test['host_response_rate'].str.replace('%', '').astype(float) / 100


# Process bathroom column into numeric column
train['bathrooms_num'] = train['bathrooms_text'].str.extract('(\d+)').astype(float)
test['bathrooms_num'] = test['bathrooms_text'].str.extract('(\d+)').astype(float)


# Convert date columns
train['host_since_years'] = ((datetime.now() - pd.to_datetime(train['host_since'])).dt.days) / 365
train['first_review_years'] = ((datetime.now() - pd.to_datetime(train['first_review'])).dt.days) / 365
train['last_review_years'] = ((datetime.now() - pd.to_datetime(train['last_review'])).dt.days) / 365
test['host_since_years'] = ((datetime.now() - pd.to_datetime(test['host_since'])).dt.days) / 365
test['first_review_years'] = (datetime.now() - pd.to_datetime(test['first_review'])).dt.days
test['last_review_years'] = ((datetime.now() - pd.to_datetime(test['last_review'])).dt.days) / 365


# Convert T/F columns
t_f_vars = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'has_availability', 'instant_bookable']
train[t_f_vars] = train[t_f_vars].replace({'f': 0, 't': 1})
test[t_f_vars] = test[t_f_vars].replace({'f': False, 't': True})


def is_shared(x):
    if str(x) == 'nan':
        return False
    else:
        if 'shared' in x:
            return 1
        else:
            return 0

        
mega_outlier = train[train['price'] > 5000].index
train.drop(mega_outlier, inplace=True)        
    
    
train['bathrooms_shared'] = train['bathrooms_text'].apply(is_shared).astype(int)
test['bathrooms_shared'] = test['bathrooms_text'].apply(is_shared).astype(int)


train_clean = train.drop(columns=['host_since', 'first_review', 'last_review', 
                                  'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'])
test_clean = test.drop(columns=['host_since', 'first_review', 'last_review', 'calculated_host_listings_count_entire_homes', 
                                'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'])


In [4]:
## Neighbourhoods with <200 observations are labeled 'Other'
neighbourhood_counts = train_clean['neighbourhood_cleansed'].value_counts()

test_only_hoods = [i for i in test_clean['neighbourhood_cleansed'].unique() 
                   if i not in neighbourhood_counts 
                   and i != 'Other']
    
other_hoods = []
for i in neighbourhood_counts.index:
    if neighbourhood_counts[i] < 200:
        other_hoods.append(i)         
 

In [5]:
def clean_hoods(row):
    if row.loc['neighbourhood_cleansed'] in other_hoods:
        row['neighbourhood_cleansed'] = 'Other'
    if row.loc['neighbourhood_cleansed'] in test_only_hoods:
        row['neighbourhood_cleansed'] = 'Other'
    return row
       
def clean_rooms(row):    
    if row.loc['room_type'] == 'Hotel room' or row.loc['room_type'] == 'Private room':
        row['room_type'] = 'Single room'
    return row

In [6]:
# Convert host_verifications 
train_clean['host_verifications_list'] = train_clean['host_verifications'].apply(lambda x: x.strip("[]").replace("'", "").split(', '))
test_clean['host_verifications_list'] = test_clean['host_verifications'].apply(lambda x: x.strip("[]").replace("'", "").split(', '))

# Make a column for number of verifications
train_clean['num_verifications']  = train_clean['host_verifications_list'].apply(len)
test_clean['num_verifications']  = test_clean['host_verifications_list'].apply(len)

# Manually create dummy variable columns for if each verification and any verification in host_verifications_list
train_clean['phone_verification']  = train_clean['host_verifications_list'].apply(lambda x: 1 if 'phone' in x else 0)
train_clean['email_verification']  = train_clean['host_verifications_list'].apply(lambda x: 1 if 'email' in x else 0)
train_clean['work_email_verification'] = train_clean['host_verifications_list'].apply(lambda x: 1 if 'work_email' in x else 0)
train_clean['any_email_verification'] = train_clean['host_verifications_list'].apply(lambda x: 1 if 'work_email' in x or 'email' in x else 0)
print(train_clean['num_verifications'].value_counts(), '\n')

test_clean['phone_verification']  = test_clean['host_verifications_list'].apply(lambda x: 1 if 'phone' in x else 0)
test_clean['email_verification']  = test_clean['host_verifications_list'].apply(lambda x: 1 if 'email' in x else 0)
test_clean['work_email_verification'] = test_clean['host_verifications_list'].apply(lambda x: 1 if 'work_email' in x else 0)
test_clean['any_email_verification'] = test_clean['host_verifications_list'].apply(lambda x: 1 if 'work_email' in x or 'email' in x else 0)
print(train_clean['num_verifications'].value_counts(), '\n')

# Group Hotel_room and Private room because of similarity characteristics and correlation
train_clean = train_clean.apply(clean_rooms, axis=1)
test_clean = test_clean.apply(clean_rooms, axis=1)
print(train_clean['room_type'].value_counts())
    
# Convert date type columns to numeric (months they have been a host)  
train_clean['host_since_months'] = train_clean['host_since_years']*12
test_clean['host_since_months'] = test_clean['host_since_years']*12
# Create reviews per month column to better standardize number of reviews
train_clean['hosts_reviews_per_month'] = train_clean['number_of_reviews']/train_clean['host_since_months']
test_clean['hosts_reviews_per_month'] = test_clean['number_of_reviews']/test_clean['host_since_months']    
    

2    3569
3    1089
1     341
Name: num_verifications, dtype: int64 

2    3569
3    1089
1     341
Name: num_verifications, dtype: int64 

Entire home/apt    3802
Single room        1153
Shared room          44
Name: room_type, dtype: int64


In [7]:
# Apply clean_hoods to group neighbourhoods with few counts
train_clean = train_clean.apply(clean_hoods, axis=1)  
test_clean = test_clean.apply(clean_hoods, axis=1)  

train_clean['neighbourhood_cleansed'].value_counts()

Other              2672
Near North Side     638
West Town           483
Lake View           355
Near West Side      338
Logan Square        261
Loop                252
Name: neighbourhood_cleansed, dtype: int64

### Related Predictors

In [8]:
grab_vars = [name for name in train_clean.columns if 'imum' in name]

grab_vars.extend(['calculated_host_listings_count', 'host_identity_verified', 'host_id'])
grab_vars.remove('minimum_nights')
grab_vars.remove('maximum_nights')


train_filter = train_clean.drop(columns=grab_vars)
test_filter = test_clean.drop(columns=grab_vars)


### Model Imputation

In [9]:
superhost_model = smf.logit(formula="host_is_superhost ~ host_total_listings_count*number_of_reviews_ltm + host_response_rate", data=train_filter).fit()

train_filter['host_is_superhost_imputed'] = superhost_model.predict(train_filter) > 0.486
train_filter['host_is_superhost'].fillna(train_filter['host_is_superhost_imputed'], inplace=True)
train_filter.drop(columns=['host_is_superhost_imputed'], inplace=True)

test_filter['host_is_superhost_imputed'] = superhost_model.predict(test_filter) > 0.486
test_filter['host_is_superhost'].fillna(test_filter['host_is_superhost_imputed'], inplace=True)
test_filter.drop(columns=['host_is_superhost_imputed'], inplace=True)


Optimization terminated successfully.
         Current function value: 0.564185
         Iterations 10


In [10]:
beds_model = smf.ols(formula='beds ~ accommodates', data=test_filter).fit()

impute_df = pd.DataFrame({'accommodates': test_filter['accommodates']})

test_filter['beds_imputed'] = beds_model.predict(impute_df)
test_filter['beds'].fillna(test_filter['beds_imputed'], inplace=True)

test_filter.drop(columns=['beds_imputed'], inplace=True)

### Naive Imputation

In [11]:
value_counts = train_filter.isna().sum()
value_counts[value_counts != 0]

review_vars = [name for name in train_filter.columns if 'review' in name]
review_corrs = train_filter[review_vars].corr()


In [12]:
temp_data_train = train_filter.dropna(how='any')

for col in temp_data_train.select_dtypes(include='number').columns:
    if train_filter.isna().sum()[col] != 0:
        train_filter[col].fillna(value=train_filter[col].median(), inplace=True) 

train_final = train_filter.copy()

In [13]:
temp_data_test = test_filter.dropna(how='any')

for col in temp_data_test.select_dtypes(include='number').columns:
    if test_filter.isna().sum()[col] != 0:
        test_filter[col].fillna(value=test_filter[col].median(), inplace=True)  

test_final = test_filter.copy()

### Binning

In [14]:
# Bin number accomodates
train_final['accommodates_bins'], bins = pd.cut(train_final['accommodates'], retbins=True, bins=6)
test_final['accommodates_bins'] = pd.cut(test['accommodates'], bins=bins)


In [15]:
# Bin number latitude and longitude
train_final['latitude_bins'], lat_bins = pd.cut(train_final['latitude'], retbins=True, bins=12)
train_final['longitude_bins'], long_bins = pd.cut(train_final['longitude'], retbins=True, bins=12)

test_final['latitude_bins'] = pd.cut(test['accommodates'], bins=lat_bins)
test_final['longitude_bins'] = pd.cut(test['accommodates'], bins=long_bins)


## 3) Developing the Model

Accomodates and room_type interaction. beds/baths and accomodates. log accomodates. 
Interactions between accomodates and room_type was selected because it reasons that while a listing that accomodates more would increase price, that will depend on what the space is like. 
Latitude and Longitude were included in addition to neighbourhood because I suspected that more central listings would have a higher price, hence the squared term to account for a curve rather than a line.

## 4) Model

In [16]:
formula = f'''price ~ neighbourhood_cleansed + I(maximum_nights**4) + 
np.log(accommodates) + accommodates*C(room_type) + 
longitude + I(longitude**2) + latitude + I(latitude**2) +
beds*accommodates + bathrooms_num*accommodates + bathrooms_num*C(bathrooms_shared)'''

model = smf.ols(formula=formula, data=train_final).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.452
Model:                            OLS   Adj. R-squared:                  0.450
Method:                 Least Squares   F-statistic:                     195.8
Date:                Sun, 18 Feb 2024   Prob (F-statistic):               0.00
Time:                        13:36:52   Log-Likelihood:                -32291.
No. Observations:                4999   AIC:                         6.463e+04
Df Residuals:                    4977   BIC:                         6.477e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     2.4784      1.066      2.325      0.020       0.389       4.568
neighbourhood_cleansed[T.Logan Square]       16.7672     13.188      1.271      0.204      -9.088      42.622
neighbourhood_cleansed[T.Loop]               30.9777     13.146      2.356      0.018       5.205      56.750
neighbourhood_cleansed[T.Near North Side]    42.2869     10.591      3.993      0.000      21.523      63.051
neighbourhood_cleansed[T.Near West Side]     15.0509     12.329      1.221      0.222      -9.120      39.221
neighbourhood_cleansed[T.Other]              -2.0547      9.415     -0.218      0.827     -20.512      16.402
neighbourhood_cleansed[T.West Town]          48.2925     11.243      4.295      0.000      26.250      70.335
C(room_type)[T.Shared room]                  -8.9815     48.849     -0.184      0.854    -104.748      86.785
C(room_type)[T.Single room]                 -53.3349     11.706     -4.556      0.000     -76.283     -30.387
C(bathrooms_shared)[T.1]                     23.6660     15.036      1.574      0.116      -5.811      53.143
I(maximum_nights ** 4)                     1.151e-11   2.96e-12      3.883      0.000     5.7e-12    1.73e-11
np.log(accommodates)                        -55.9926     14.179     -3.949      0.000     -83.789     -28.196
accommodates                                 18.2658      3.747      4.875      0.000      10.920      25.611
accommodates:C(room_type)[T.Shared room]     -0.7162     18.123     -0.040      0.968     -36.246      34.813
accommodates:C(room_type)[T.Single room]     24.4250      2.678      9.119      0.000      19.174      29.676
longitude                                  -108.6490     46.722     -2.325      0.020    -200.245     -17.053
I(longitude ** 2)                            -1.7894      0.647     -2.765      0.006      -3.058      -0.521
latitude                                     51.8928     22.305      2.327      0.020       8.165      95.620
I(latitude ** 2)                              1.1875      0.531      2.237      0.025       0.147       2.228
beds                                         20.0975      4.829      4.162      0.000      10.630      29.565
beds:accommodates                            -3.1681      0.457     -6.927      0.000      -4.065      -2.272
bathrooms_num                                 8.8563      9.305      0.952      0.341      -9.386      27.099
bathrooms_num:C(bathrooms_shared)[T.1]      -45.2373     10.236     -4.419      0.000     -65.305     -25.170
bathrooms_num:accommodates                   12.3970      0.989     12.538      0.000      10.459      14.335
==============================================================================
Omnibus:                     8202.

In [17]:
# Predicted values
trying_pred = model.predict(train_final)

# Evaluate
rmse = mean_squared_error(train_final['price'], trying_pred, squared=False)
mae = mean_absolute_error(train_final['price'], trying_pred)
mae_rmse_diff = rmse - mae 

print(f'''Root Mean Squared Error (RMSE): {round(rmse, 3)}\nMean Absolute Error (MAE): {round(mae, 3)}\nDiff: {mae_rmse_diff}''')


Root Mean Squared Error (RMSE): 154.57
Mean Absolute Error (MAE): 67.118
Diff: 87.45230123668235


In [18]:
predicted_values = pd.DataFrame(model.predict(test_final))
predicted_values = predicted_values.merge(test_final['id'], left_index=True, right_index=True).set_index('id').rename(columns={0:'predicted'})

predicted_values.to_csv('liner_model_results.csv') 